In [1]:
import pandas as pd
import os
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from fuzzywuzzy import process

# Get the current working directory
current_dir = os.getcwd()

# Define the path to the dataset directory relative to the current working directory
dataset_dir = os.path.join(current_dir, 'ml-latest-small')

# Load MovieLens data
movies = pd.read_csv(os.path.join(dataset_dir, 'movies.csv'))
ratings = pd.read_csv(os.path.join(dataset_dir, 'ratings.csv'))

# Merge movies and ratings datasets
data = pd.merge(ratings, movies, on='movieId')

# Create a user-movie matrix
user_movie_matrix = data.pivot_table(index='userId', columns='title', values='rating')

# Fill NaN values with 0
user_movie_matrix = user_movie_matrix.fillna(0)

# Compute cosine similarity between users
user_similarity = cosine_similarity(user_movie_matrix)
user_similarity_matrix = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

# Function to get movie recommendations based on user's liked movies
def get_recommendations_from_likes(liked_movies, num_recommendations=5):
    # Create a new user profile based on the liked movies
    new_user_profile = pd.Series(0, index=user_movie_matrix.columns)
    for movie in liked_movies:
        if movie in new_user_profile.index:
            new_user_profile[movie] = 5  # Assign the highest rating to liked movies

    # Compute similarity between the new user and existing users
    new_user_similarity = cosine_similarity([new_user_profile], user_movie_matrix)[0]

    # Compute weighted ratings
    weighted_ratings = np.dot(new_user_similarity, user_movie_matrix) / np.array([np.abs(new_user_similarity).sum()])
    recommendations = pd.Series(weighted_ratings, index=user_movie_matrix.columns).sort_values(ascending=False).head(num_recommendations + len(liked_movies))

    # Filter out already liked movies
    recommendations = recommendations[~recommendations.index.isin(liked_movies)]
    
    return recommendations.index.tolist()[:num_recommendations]

# Function to get the closest movie title matches
def get_closest_matches(query, choices, n=1):
    return process.extract(query, choices, limit=n)

# Get user input
liked_movies_input = input("Enter the movies you liked, separated by commas: ").split(',')

# Clean up the user input
liked_movies_input = [movie.strip() for movie in liked_movies_input]

# Get the closest matches for each input movie
all_movies = movies['title'].tolist()
liked_movies = []
for movie in liked_movies_input:
    match = get_closest_matches(movie, all_movies, n=1)
    if match:
        matched_movie = match[0][0]
        confirm = input(f"Did you mean '{matched_movie}'? (yes/no): ").strip().lower()
        if confirm == 'yes':
            liked_movies.append(matched_movie)

# Get recommendations
recommendations = get_recommendations_from_likes(liked_movies)
print(f"Recommendations based on your liked movies: {recommendations}")


Recommendations based on your liked movies: ['Forrest Gump (1994)', 'Matrix, The (1999)', 'Fight Club (1999)', 'Star Wars: Episode IV - A New Hope (1977)', 'Shawshank Redemption, The (1994)']
